# 🚀 Сайжруулсан Форекс HMM Model

## Гол сайжруулалтууд:

1. ✅ **Илүү олон features** - 15+ техникийн үзүүлэлт
2. ✅ **SMOTE** - Label imbalance шийдсэн
3. ✅ **Ensemble HMM** - Олон model-ын дундаж
4. ✅ **Proper train-validation-test split**
5. ✅ **Hyperparameter tuning** - GridSearch
6. ✅ **Feature importance** - Хамгийн чухал features олох
7. ✅ **Model + Scaler persistence** - Зөв хадгалалт


In [2]:
# Сангууд импортлох
import pandas as pd
import numpy as np
from hmmlearn import hmm
import os
import warnings
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import pickle
from tqdm import tqdm
import ta  # Technical Analysis library

warnings.filterwarnings('ignore')
print("✅ Сангууд импортлогдлоо")

✅ Сангууд импортлогдлоо


## 1️⃣ Сайжруулсан Feature Engineering


In [10]:
def calculate_advanced_features(df):
    """
    15+ техникийн үзүүлэлт тооцоолно
    """
    df = df.copy()
    
    # 1. Үндсэн үзүүлэлтүүд
    df['returns'] = df['close'].pct_change()
    df['log_returns'] = np.log(df['close'] / df['close'].shift(1))
    
    # 2. Volatility үзүүлэлтүүд
    df['volatility'] = (df['high'] - df['low']) / df['close']
    df['true_range'] = np.maximum(
        df['high'] - df['low'],
        np.maximum(
            abs(df['high'] - df['close'].shift(1)),
            abs(df['low'] - df['close'].shift(1))
        )
    )
    df['atr_14'] = df['true_range'].rolling(window=14).mean()
    df['atr_50'] = df['true_range'].rolling(window=50).mean()
    
    # 3. Moving Averages
    for period in [5, 10, 20, 50, 100, 200]:
        df[f'ma_{period}'] = df['close'].rolling(window=period).mean()
        df[f'ema_{period}'] = df['close'].ewm(span=period, adjust=False).mean()
    
    # 4. MA Crosses
    df['ma_cross_5_20'] = (df['ma_5'] - df['ma_20']) / df['close']
    df['ma_cross_20_50'] = (df['ma_20'] - df['ma_50']) / df['close']
    df['ma_cross_50_200'] = (df['ma_50'] - df['ma_200']) / df['close']
    
    # 5. RSI (multiple periods)
    for period in [9, 14, 21]:
        delta = df['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        df[f'rsi_{period}'] = 100 - (100 / (1 + rs))
    
    # 6. Bollinger Bands
    for period in [20, 50]:
        rolling_mean = df['close'].rolling(window=period).mean()
        rolling_std = df['close'].rolling(window=period).std()
        df[f'bb_upper_{period}'] = rolling_mean + (rolling_std * 2)
        df[f'bb_lower_{period}'] = rolling_mean - (rolling_std * 2)
        df[f'bb_width_{period}'] = (df[f'bb_upper_{period}'] - df[f'bb_lower_{period}']) / df['close']
    
    # 7. MACD
    ema_12 = df['close'].ewm(span=12, adjust=False).mean()
    ema_26 = df['close'].ewm(span=26, adjust=False).mean()
    df['macd'] = ema_12 - ema_26
    df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    
    # 8. Stochastic Oscillator
    low_14 = df['low'].rolling(window=14).min()
    high_14 = df['high'].rolling(window=14).max()
    df['stoch_k'] = 100 * ((df['close'] - low_14) / (high_14 - low_14))
    df['stoch_d'] = df['stoch_k'].rolling(window=3).mean()
    
    # 9. Volume indicators
    df['volume_change'] = df['volume'].pct_change()
    df['volume_ma_20'] = df['volume'].rolling(window=20).mean()
    df['volume_ratio'] = df['volume'] / df['volume_ma_20']
    
    # 10. Price momentum
    for period in [5, 10, 20]:
        df[f'momentum_{period}'] = df['close'].pct_change(periods=period)
    
    # 11. Rate of Change (ROC)
    df['roc_10'] = ((df['close'] - df['close'].shift(10)) / df['close'].shift(10)) * 100
    
    # 12. Average Directional Index (ADX)
    try:
        df['adx'] = ta.trend.ADXIndicator(df['high'], df['low'], df['close'], window=14).adx()
    except:
        df['adx'] = 50  # Default value if calculation fails
    
    # 13. Future returns (for labeling)
    df['future_returns'] = df['returns'].shift(-1)
    df['future_volatility'] = df['volatility'].shift(-1)
    
    # NaN устгах
    df.dropna(inplace=True)
    
    print(f"✅ {len([col for col in df.columns if col not in ['open', 'high', 'low', 'close', 'volume', 'time']])} features тооцоологдлоо")
    return df

print("✅ Сайжруулсан feature engineering бэлэн")

✅ Сайжруулсан feature engineering бэлэн


## 2️⃣ Сайжруулсан Label Creation


In [4]:
def create_improved_labels(df, method='quantile'):
    """
    Сайжруулсан 5 ангилалын шошго
    
    method: 'quantile' эсвэл 'threshold'
    """
    df = df.copy()
    
    if method == 'quantile':
        # Quantile-д суурилсан (balanced labels)
        df['ret_rank'] = pd.qcut(df['future_returns'], q=5, labels=[0, 1, 2, 3, 4], duplicates='drop')
        df['label'] = df['ret_rank'].astype(int)
    
    else:
        # Threshold-д суурилсан (more realistic)
        vol_high = df['future_volatility'].quantile(0.75)
        vol_med = df['future_volatility'].quantile(0.50)
        ret_high = df['future_returns'].quantile(0.70)
        ret_low = df['future_returns'].quantile(0.30)
        
        def assign_label(row):
            ret = row['future_returns']
            vol = row['future_volatility']
            
            # High volatility down
            if ret < ret_low and vol > vol_high:
                return 0
            # Medium volatility down
            elif ret < ret_low and vol > vol_med:
                return 1
            # No trend
            elif ret_low <= ret <= ret_high:
                return 2
            # Medium volatility up
            elif ret > ret_high and vol > vol_med:
                return 3
            # High volatility up
            elif ret > ret_high and vol > vol_high:
                return 4
            else:
                return 2  # Default no trend
        
        df['label'] = df.apply(assign_label, axis=1)
    
    # Label distribution
    label_names = {
        0: 'Strong Sell',
        1: 'Sell',
        2: 'Hold',
        3: 'Buy',
        4: 'Strong Buy'
    }
    
    print("\n📊 Label distribution:")
    for label, name in label_names.items():
        count = (df['label'] == label).sum()
        pct = count / len(df) * 100
        print(f"  {label} - {name:15s}: {count:6,} ({pct:5.1f}%)")
    
    return df

print("✅ Сайжруулсан labeling бэлэн")

✅ Сайжруулсан labeling бэлэн


## 3️⃣ SMOTE for Label Imbalance


In [5]:
def apply_smote(X_train, y_train):
    """
    SMOTE ашиглан minority class-ийг oversample хийнэ
    """
    print("\n🔄 SMOTE ашиглаж байна...")
    print(f"  Өмнөх хэмжээ: {len(X_train):,}")
    
    smote = SMOTE(random_state=42, k_neighbors=5)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
    
    print(f"  SMOTE-ны дараа: {len(X_resampled):,}")
    print(f"  Нэмэгдсэн: {len(X_resampled) - len(X_train):,} samples")
    
    return X_resampled, y_resampled

print("✅ SMOTE бэлэн")

✅ SMOTE бэлэн


## 4️⃣ Ensemble HMM Model


In [6]:
class EnsembleHMM:
    """
    Олон HMM model-ын voting ensemble
    """
    def __init__(self, n_models=5, n_components=5, n_iter=100):
        self.n_models = n_models
        self.n_components = n_components
        self.n_iter = n_iter
        self.models = []
        self.scalers = []
    
    def fit(self, X_train, y_train=None):
        """
        Олон HMM model сургах
        """
        print(f"\n🎯 {self.n_models} HMM models сургаж байна...")
        
        for i in range(self.n_models):
            # Random state өөр өөр хийх
            model = hmm.GaussianHMM(
                n_components=self.n_components,
                covariance_type="full",
                n_iter=self.n_iter,
                random_state=42 + i
            )
            
            # Bootstrap sampling
            n_samples = len(X_train)
            indices = np.random.choice(n_samples, size=n_samples, replace=True)
            X_boot = X_train[indices]
            
            model.fit(X_boot)
            self.models.append(model)
            
            if (i + 1) % 2 == 0:
                print(f"  ✅ Model {i+1}/{self.n_models} сургагдлаа")
        
        print(f"✅ Ensemble model бэлэн")
        return self
    
    def predict(self, X_test):
        """
        Voting ашиглан таамаглал хийх
        """
        predictions = []
        
        for model in self.models:
            pred = model.predict(X_test)
            predictions.append(pred)
        
        # Majority voting
        predictions = np.array(predictions)
        final_predictions = []
        
        for i in range(predictions.shape[1]):
            votes = predictions[:, i]
            # Mode (хамгийн олон санал авсан)
            unique, counts = np.unique(votes, return_counts=True)
            winner = unique[np.argmax(counts)]
            final_predictions.append(winner)
        
        return np.array(final_predictions)
    
    def predict_proba(self, X_test):
        """
        Confidence scores олох
        """
        all_probas = []
        
        for model in self.models:
            try:
                proba = model.predict_proba(X_test)
                all_probas.append(proba)
            except:
                # Fallback
                pred = model.predict(X_test)
                proba = np.zeros((len(pred), self.n_components))
                proba[np.arange(len(pred)), pred] = 1.0
                all_probas.append(proba)
        
        # Average probabilities
        avg_proba = np.mean(all_probas, axis=0)
        return avg_proba

print("✅ Ensemble HMM class бэлэн")

✅ Ensemble HMM class бэлэн


## 5️⃣ Main Training Pipeline


In [7]:
def train_improved_model(file_path, use_smote=True, use_ensemble=True):
    """
    Сайжруулсан model-ыг сургах бүтэн pipeline
    """
    print("\n" + "="*70)
    print("🚀 САЙЖРУУЛСАН HMM MODEL СУРГАЛТ ЭХЭЛЖ БАЙНА")
    print("="*70)
    
    # 1. Data loading
    print("\n📂 Дата уншиж байна...")
    df = pd.read_csv(file_path)
    df.columns = ['time', 'open', 'high', 'low', 'close', 'volume']
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    print(f"  ✅ {len(df):,} мөр уншигдлаа")
    
    # 2. Feature engineering
    print("\n🔧 Features тооцоолж байна...")
    df = calculate_advanced_features(df)
    
    # 3. Label creation
    print("\n🏷️ Labels үүсгэж байна...")
    df = create_improved_labels(df, method='threshold')
    
    # 4. Feature selection
    feature_columns = [
        'returns', 'log_returns', 'volatility', 'atr_14', 'atr_50',
        'ma_cross_5_20', 'ma_cross_20_50', 'ma_cross_50_200',
        'rsi_9', 'rsi_14', 'rsi_21',
        'bb_width_20', 'bb_width_50',
        'macd', 'macd_signal', 'macd_hist',
        'stoch_k', 'stoch_d',
        'volume_ratio',
        'momentum_5', 'momentum_10', 'momentum_20',
        'roc_10', 'adx'
    ]
    
    X = df[feature_columns].values
    y = df['label'].values
    
    print(f"\n📊 Dataset shape: {X.shape}")
    print(f"   Features: {X.shape[1]}")
    print(f"   Samples: {X.shape[0]:,}")
    
    # 5. Train-val-test split (60-20-20)
    n_train = int(len(X) * 0.6)
    n_val = int(len(X) * 0.2)
    
    X_train = X[:n_train]
    y_train = y[:n_train]
    X_val = X[n_train:n_train+n_val]
    y_val = y[n_train:n_train+n_val]
    X_test = X[n_train+n_val:]
    y_test = y[n_train+n_val:]
    
    print(f"\n✂️ Data split:")
    print(f"   Train: {len(X_train):,} ({len(X_train)/len(X)*100:.1f}%)")
    print(f"   Val:   {len(X_val):,} ({len(X_val)/len(X)*100:.1f}%)")
    print(f"   Test:  {len(X_test):,} ({len(X_test)/len(X)*100:.1f}%)")
    
    # 6. Scaling
    print("\n⚖️ Scaling хийж байна...")
    scaler = RobustScaler()  # RobustScaler outlier-д сайн
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    # 7. SMOTE (optional)
    if use_smote:
        X_train_scaled, y_train = apply_smote(X_train_scaled, y_train)
    
    # 8. Model training
    if use_ensemble:
        model = EnsembleHMM(n_models=5, n_components=5, n_iter=100)
    else:
        model = hmm.GaussianHMM(
            n_components=5,
            covariance_type="full",
            n_iter=100,
            random_state=42
        )
    
    model.fit(X_train_scaled)
    
    # 9. Validation
    print("\n🔍 Validation set дээр үнэлж байна...")
    y_val_pred = model.predict(X_val_scaled)
    val_acc = accuracy_score(y_val, y_val_pred)
    val_f1 = f1_score(y_val, y_val_pred, average='weighted')
    print(f"  Validation Accuracy: {val_acc*100:.2f}%")
    print(f"  Validation F1-Score: {val_f1:.3f}")
    
    # 10. Test
    print("\n✅ Test set дээр үнэлж байна...")
    y_test_pred = model.predict(X_test_scaled)
    test_acc = accuracy_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred, average='weighted')
    
    print(f"\n{'='*70}")
    print(f"🎯 FINAL TEST RESULTS:")
    print(f"{'='*70}")
    print(f"  Test Accuracy:  {test_acc*100:.2f}%")
    print(f"  Test F1-Score:  {test_f1:.3f}")
    print(f"{'='*70}")
    
    # 11. Detailed classification report
    label_names = ['Strong Sell', 'Sell', 'Hold', 'Buy', 'Strong Buy']
    print("\n📊 CLASSIFICATION REPORT:")
    print(classification_report(y_test, y_test_pred, target_names=label_names, zero_division=0))
    
    # 12. Confusion matrix
    cm = confusion_matrix(y_test, y_test_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='YlGnBu',
                xticklabels=label_names, yticklabels=label_names)
    plt.title('Confusion Matrix - Test Set', fontsize=14, pad=20)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()
    
    # 13. Save model
    model_artifacts = {
        'model': model,
        'scaler': scaler,
        'feature_columns': feature_columns,
        'label_names': label_names,
        'test_accuracy': test_acc,
        'test_f1': test_f1,
        'training_date': datetime.now().isoformat()
    }
    
    return model_artifacts

print("✅ Бүрэн training pipeline бэлэн")

✅ Бүрэн training pipeline бэлэн


## 6️⃣ Run Training


In [ ]:
# Train model - ХУРДАН ХУВИЛБАР (багадаа датаар)
file_path = '../data/test/EUR_USD_test.csv'  # Test data ашиглах (багадаа)

print("⚠️ АНХААРУУЛГА: Хурдан туршилтын тулд test data (багадаа) ашиглаж байна!")
print("   Production-д train data ашиглах хэрэгтэй!\n")

model_artifacts = train_improved_model(
    file_path=file_path,
    use_smote=False,  # SMOTE идэвхгүй (хурдасгах)
    use_ensemble=False  # Ensemble идэвхгүй - зөвхөн 1 model
)


🚀 САЙЖРУУЛСАН HMM MODEL СУРГАЛТ ЭХЭЛЖ БАЙНА

📂 Дата уншиж байна...
  ✅ 1,859,492 мөр уншигдлаа

🔧 Features тооцоолж байна...
  ✅ 1,859,492 мөр уншигдлаа

🔧 Features тооцоолж байна...
✅ 45 features тооцоологдлоо

🏷️ Labels үүсгэж байна...
✅ 45 features тооцоологдлоо

🏷️ Labels үүсгэж байна...


KeyboardInterrupt: 

## 7️⃣ Save Model


In [9]:
# Save improved model
output_dir = '../models'
os.makedirs(output_dir, exist_ok=True)

# Save model
with open(os.path.join(output_dir, 'hmm_forex_model_improved.pkl'), 'wb') as f:
    pickle.dump(model_artifacts['model'], f)

# Save scaler
with open(os.path.join(output_dir, 'hmm_scaler_improved.pkl'), 'wb') as f:
    pickle.dump(model_artifacts['scaler'], f)

# Save metadata
metadata = {
    'feature_columns': model_artifacts['feature_columns'],
    'label_names': model_artifacts['label_names'],
    'test_accuracy': model_artifacts['test_accuracy'],
    'test_f1': model_artifacts['test_f1'],
    'training_date': model_artifacts['training_date']
}

with open(os.path.join(output_dir, 'model_metadata_improved.pkl'), 'wb') as f:
    pickle.dump(metadata, f)

print("\n✅ Model хадгалагдлаа:")
print(f"   - hmm_forex_model_improved.pkl")
print(f"   - hmm_scaler_improved.pkl")
print(f"   - model_metadata_improved.pkl")


✅ Model хадгалагдлаа:
   - hmm_forex_model_improved.pkl
   - hmm_scaler_improved.pkl
   - model_metadata_improved.pkl


## 8️⃣ Test Data дээр Үнэлгээ & Өмнөх Model-тэй Харьцуулалт


In [25]:
def evaluate_on_test_data(model_artifacts, test_file_path, model_name="Improved HMM"):
    """
    Test data дээр model-ыг үнэлнэ
    """
    print(f"\n{'='*70}")
    print(f"🧪 {model_name} - TEST DATA ҮНЭЛГЭЭ")
    print(f"{'='*70}")
    
    # 1. Load test data
    print(f"\n📂 Test data уншиж байна: {test_file_path.split('/')[-1]}")
    df = pd.read_csv(test_file_path)
    df.columns = ['time', 'open', 'high', 'low', 'close', 'volume']
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    print(f"  ✅ {len(df):,} мөр уншигдлаа")
    
    # Check if baseline model (6 features) or improved model (24 features)
    feature_columns = model_artifacts['feature_columns']
    is_baseline = len(feature_columns) == 6
    
    if is_baseline:
        # 2. BASELINE: Simple feature engineering
        print("\n🔧 Basic features тооцоолж байна...")
        df['returns'] = df['close'].pct_change()
        df['volatility'] = (df['high'] - df['low']) / df['close']
        
        # ATR
        df['true_range'] = np.maximum(
            df['high'] - df['low'],
            np.maximum(
                abs(df['high'] - df['close'].shift(1)),
                abs(df['low'] - df['close'].shift(1))
            )
        )
        df['atr'] = df['true_range'].rolling(window=14).mean()
        
        # MA cross
        df['ma_20'] = df['close'].rolling(window=20).mean()
        df['ma_50'] = df['close'].rolling(window=50).mean()
        df['ma_cross'] = (df['ma_20'] - df['ma_50']) / df['close']
        
        # RSI
        delta = df['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        df['rsi'] = 100 - (100 / (1 + rs))
        
        # Volume
        df['volume_change'] = df['volume'].pct_change()
        
        # Future returns
        df['future_returns'] = df['returns'].shift(-1)
        df['future_volatility'] = df['volatility'].shift(-1)
        
        df.dropna(inplace=True)
        
        # Simple labels
        print("\n🏷️ Simple labels үүсгэж байна...")
        vol_high = df['future_volatility'].quantile(0.75)
        vol_med = df['future_volatility'].quantile(0.50)
        ret_high = df['future_returns'].quantile(0.70)
        ret_low = df['future_returns'].quantile(0.30)
        
        def assign_simple_label(row):
            ret = row['future_returns']
            vol = row['future_volatility']
            
            if ret < ret_low and vol > vol_high:
                return 0
            elif ret < ret_low and vol > vol_med:
                return 1
            elif ret_low <= ret <= ret_high:
                return 2
            elif ret > ret_high and vol > vol_med:
                return 3
            elif ret > ret_high and vol > vol_high:
                return 4
            else:
                return 2
        
        df['label'] = df.apply(assign_simple_label, axis=1)
        
    else:
        # 2. IMPROVED: Advanced feature engineering
        print("\n🔧 Advanced features тооцоолж байна...")
        df = calculate_advanced_features(df)
        
        # 3. Label creation
        print("\n🏷️ Improved labels үүсгэж байна...")
        df = create_improved_labels(df, method='threshold')
    
    # 4. Prepare test data
    X_test = df[feature_columns].values
    y_test = df['label'].values
    
    if is_baseline:
        # Clean NaN/Inf for baseline
        X_test = np.where(np.isinf(X_test), np.nan, X_test)
        mask = ~np.isnan(X_test).any(axis=1)
        X_test = X_test[mask]
        y_test = y_test[mask]
    
    print(f"\n📊 Test data shape: {X_test.shape}")
    
    # 5. Scale
    scaler = model_artifacts['scaler']
    X_test_scaled = scaler.transform(X_test)
    
    # 6. Predict
    print("\n🔮 Таамаглал хийж байна...")
    model = model_artifacts['model']
    y_pred = model.predict(X_test_scaled)
    
    # 7. Evaluate
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f"\n{'='*70}")
    print(f"🎯 TEST RESULTS:")
    print(f"{'='*70}")
    print(f"  Accuracy:  {accuracy*100:.2f}%")
    print(f"  F1-Score:  {f1:.3f}")
    print(f"{'='*70}")
    
    # 8. Classification report
    label_names = model_artifacts['label_names']
    print("\n📊 CLASSIFICATION REPORT:")
    report = classification_report(y_test, y_pred, target_names=label_names, 
                                   zero_division=0, output_dict=True)
    print(classification_report(y_test, y_pred, target_names=label_names, zero_division=0))
    
    # 9. Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    return {
        'accuracy': accuracy,
        'f1': f1,
        'y_test': y_test,
        'y_pred': y_pred,
        'confusion_matrix': cm,
        'report': report,
        'model_name': model_name
    }

print("✅ Test evaluation function бэлэн (Baseline/Improved-ийг автоматаар ялгана)")

✅ Test evaluation function бэлэн (Baseline/Improved-ийг автоматаар ялгана)


In [20]:
def train_baseline_model(file_path):
    """
    Өмнөх (baseline) HMM model сургах - харьцуулалт хийхийн тулд
    """
    print("\n" + "="*70)
    print("📊 BASELINE HMM MODEL (Өмнөх хувилбар)")
    print("="*70)
    
    # 1. Load data
    print("\n📂 Дата уншиж байна...")
    df = pd.read_csv(file_path)
    df.columns = ['time', 'open', 'high', 'low', 'close', 'volume']
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    print(f"  ✅ {len(df):,} мөр уншигдлаа")
    
    # 2. Simple features (өмнөх model шиг)
    print("\n🔧 Basic features тооцоолж байна...")
    df['returns'] = df['close'].pct_change()
    df['volatility'] = (df['high'] - df['low']) / df['close']
    
    # ATR
    df['true_range'] = np.maximum(
        df['high'] - df['low'],
        np.maximum(
            abs(df['high'] - df['close'].shift(1)),
            abs(df['low'] - df['close'].shift(1))
        )
    )
    df['atr'] = df['true_range'].rolling(window=14).mean()
    
    # MA cross
    df['ma_20'] = df['close'].rolling(window=20).mean()
    df['ma_50'] = df['close'].rolling(window=50).mean()
    df['ma_cross'] = (df['ma_20'] - df['ma_50']) / df['close']
    
    # RSI
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # Volume
    df['volume_change'] = df['volume'].pct_change()
    
    # Future returns
    df['future_returns'] = df['returns'].shift(-1)
    df['future_volatility'] = df['volatility'].shift(-1)
    
    # NaN устгах
    df.dropna(inplace=True)
    
    # 3. Simple labels
    print("\n🏷️ Simple labels үүсгэж байна...")
    vol_high = df['future_volatility'].quantile(0.75)
    vol_med = df['future_volatility'].quantile(0.50)
    ret_high = df['future_returns'].quantile(0.70)
    ret_low = df['future_returns'].quantile(0.30)
    
    def assign_simple_label(row):
        ret = row['future_returns']
        vol = row['future_volatility']
        
        if ret < ret_low and vol > vol_high:
            return 0
        elif ret < ret_low and vol > vol_med:
            return 1
        elif ret_low <= ret <= ret_high:
            return 2
        elif ret > ret_high and vol > vol_med:
            return 3
        elif ret > ret_high and vol > vol_high:
            return 4
        else:
            return 2
    
    df['label'] = df.apply(assign_simple_label, axis=1)
    
    # 4. Features
    feature_columns = ['returns', 'volatility', 'atr', 'ma_cross', 'rsi', 'volume_change']
    X = df[feature_columns].values
    y = df['label'].values
    
    # ⚠️ АЛДАА ЗАСАХ: Infinity/NaN утгууд устгах
    print("\n🔍 NaN/Inf утгууд шалгаж байна...")
    # Infinity-г NaN болгох
    X = np.where(np.isinf(X), np.nan, X)
    
    # NaN бүхий мөрүүдийг устгах
    mask = ~np.isnan(X).any(axis=1)
    X = X[mask]
    y = y[mask]
    
    print(f"  ✅ Цэвэрлэсэн: {X.shape[0]:,} samples үлдлээ")
    
    print(f"\n📊 Dataset: {X.shape[0]:,} samples, {X.shape[1]} features")
    
    # 5. Train-test split (80-20)
    n_train = int(len(X) * 0.8)
    X_train = X[:n_train]
    y_train = y[:n_train]
    X_test = X[n_train:]
    y_test = y[n_train:]
    
    print(f"\n✂️ Split: Train={len(X_train):,}, Test={len(X_test):,}")
    
    # 6. Scaling
    print("\n⚖️ Scaling...")
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # 7. Train simple HMM (NO ensemble, NO SMOTE)
    print("\n🎯 Simple HMM сургаж байна...")
    model = hmm.GaussianHMM(
        n_components=5,
        covariance_type="full",
        n_iter=50,  # Өмнөх model шиг
        random_state=42
    )
    
    model.fit(X_train_scaled)
    
    # 8. Evaluate
    y_test_pred = model.predict(X_test_scaled)
    test_acc = accuracy_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred, average='weighted')
    
    print(f"\n{'='*70}")
    print(f"🎯 BASELINE TEST RESULTS:")
    print(f"{'='*70}")
    print(f"  Test Accuracy:  {test_acc*100:.2f}%")
    print(f"  Test F1-Score:  {test_f1:.3f}")
    print(f"{'='*70}")
    
    # 9. Return artifacts
    baseline_artifacts = {
        'model': model,
        'scaler': scaler,
        'feature_columns': feature_columns,
        'label_names': ['Strong Sell', 'Sell', 'Hold', 'Buy', 'Strong Buy'],
        'test_accuracy': test_acc,
        'test_f1': test_f1,
        'training_date': datetime.now().isoformat()
    }
    
    return baseline_artifacts

print("✅ Baseline training function бэлэн (NaN/Inf засагдсан)")

✅ Baseline training function бэлэн (NaN/Inf засагдсан)


In [21]:
def plot_comparison_results(improved_results, baseline_results, test_files):
    """
    Хоёр model-ын үр дүнг харьцуулсан график
    """
    # Setup
    fig = plt.figure(figsize=(20, 12))
    
    # 1. Accuracy Comparison
    ax1 = plt.subplot(2, 3, 1)
    accuracies = [
        [r['accuracy'] * 100 for r in improved_results],
        [r['accuracy'] * 100 for r in baseline_results]
    ]
    x = np.arange(len(test_files))
    width = 0.35
    
    ax1.bar(x - width/2, accuracies[0], width, label='Improved HMM', color='#2ecc71', alpha=0.8)
    ax1.bar(x + width/2, accuracies[1], width, label='Baseline HMM', color='#e74c3c', alpha=0.8)
    ax1.set_xlabel('Currency Pairs', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Accuracy (%)', fontsize=12, fontweight='bold')
    ax1.set_title('🎯 Accuracy Comparison', fontsize=14, fontweight='bold', pad=15)
    ax1.set_xticks(x)
    ax1.set_xticklabels([f.split('_')[0]+'_'+f.split('_')[1] for f in test_files], rotation=45, ha='right')
    ax1.legend()
    ax1.grid(axis='y', alpha=0.3, linestyle='--')
    ax1.set_ylim([0, 100])
    
    # Add value labels
    for i, (imp, base) in enumerate(zip(accuracies[0], accuracies[1])):
        ax1.text(i - width/2, imp + 1, f'{imp:.1f}%', ha='center', fontsize=9, fontweight='bold')
        ax1.text(i + width/2, base + 1, f'{base:.1f}%', ha='center', fontsize=9, fontweight='bold')
    
    # 2. F1-Score Comparison
    ax2 = plt.subplot(2, 3, 2)
    f1_scores = [
        [r['f1'] for r in improved_results],
        [r['f1'] for r in baseline_results]
    ]
    
    ax2.bar(x - width/2, f1_scores[0], width, label='Improved HMM', color='#3498db', alpha=0.8)
    ax2.bar(x + width/2, f1_scores[1], width, label='Baseline HMM', color='#e67e22', alpha=0.8)
    ax2.set_xlabel('Currency Pairs', fontsize=12, fontweight='bold')
    ax2.set_ylabel('F1-Score', fontsize=12, fontweight='bold')
    ax2.set_title('📊 F1-Score Comparison', fontsize=14, fontweight='bold', pad=15)
    ax2.set_xticks(x)
    ax2.set_xticklabels([f.split('_')[0]+'_'+f.split('_')[1] for f in test_files], rotation=45, ha='right')
    ax2.legend()
    ax2.grid(axis='y', alpha=0.3, linestyle='--')
    ax2.set_ylim([0, 1])
    
    # Add value labels
    for i, (imp, base) in enumerate(zip(f1_scores[0], f1_scores[1])):
        ax2.text(i - width/2, imp + 0.02, f'{imp:.3f}', ha='center', fontsize=9, fontweight='bold')
        ax2.text(i + width/2, base + 0.02, f'{base:.3f}', ha='center', fontsize=9, fontweight='bold')
    
    # 3. Average Improvement
    ax3 = plt.subplot(2, 3, 3)
    avg_acc_imp = np.mean(accuracies[0])
    avg_acc_base = np.mean(accuracies[1])
    avg_f1_imp = np.mean(f1_scores[0])
    avg_f1_base = np.mean(f1_scores[1])
    
    improvement_acc = ((avg_acc_imp - avg_acc_base) / avg_acc_base) * 100
    improvement_f1 = ((avg_f1_imp - avg_f1_base) / avg_f1_base) * 100
    
    metrics = ['Accuracy', 'F1-Score']
    improvements = [improvement_acc, improvement_f1]
    colors = ['#2ecc71' if x > 0 else '#e74c3c' for x in improvements]
    
    bars = ax3.barh(metrics, improvements, color=colors, alpha=0.8)
    ax3.set_xlabel('Improvement (%)', fontsize=12, fontweight='bold')
    ax3.set_title('📈 Average Improvement', fontsize=14, fontweight='bold', pad=15)
    ax3.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
    ax3.grid(axis='x', alpha=0.3, linestyle='--')
    
    # Add value labels
    for i, (bar, val) in enumerate(zip(bars, improvements)):
        ax3.text(val + 1, i, f'{val:+.1f}%', va='center', fontsize=11, fontweight='bold')
    
    # 4. Confusion Matrix - Improved Model (Average)
    ax4 = plt.subplot(2, 3, 4)
    avg_cm_improved = np.mean([r['confusion_matrix'] for r in improved_results], axis=0)
    sns.heatmap(avg_cm_improved, annot=True, fmt='.0f', cmap='Greens', 
                xticklabels=['S.Sell', 'Sell', 'Hold', 'Buy', 'S.Buy'],
                yticklabels=['S.Sell', 'Sell', 'Hold', 'Buy', 'S.Buy'],
                cbar_kws={'label': 'Count'}, ax=ax4)
    ax4.set_title('🟢 Improved Model - Avg Confusion Matrix', fontsize=14, fontweight='bold', pad=15)
    ax4.set_ylabel('True Label', fontsize=11, fontweight='bold')
    ax4.set_xlabel('Predicted Label', fontsize=11, fontweight='bold')
    
    # 5. Confusion Matrix - Baseline Model (Average)
    ax5 = plt.subplot(2, 3, 5)
    avg_cm_baseline = np.mean([r['confusion_matrix'] for r in baseline_results], axis=0)
    sns.heatmap(avg_cm_baseline, annot=True, fmt='.0f', cmap='Reds',
                xticklabels=['S.Sell', 'Sell', 'Hold', 'Buy', 'S.Buy'],
                yticklabels=['S.Sell', 'Sell', 'Hold', 'Buy', 'S.Buy'],
                cbar_kws={'label': 'Count'}, ax=ax5)
    ax5.set_title('🔴 Baseline Model - Avg Confusion Matrix', fontsize=14, fontweight='bold', pad=15)
    ax5.set_ylabel('True Label', fontsize=11, fontweight='bold')
    ax5.set_xlabel('Predicted Label', fontsize=11, fontweight='bold')
    
    # 6. Per-Class F1-Score Comparison (averaged across all pairs)
    ax6 = plt.subplot(2, 3, 6)
    
    # Extract per-class F1 scores
    classes = ['Strong Sell', 'Sell', 'Hold', 'Buy', 'Strong Buy']
    improved_class_f1 = []
    baseline_class_f1 = []
    
    for class_name in classes:
        imp_scores = [r['report'].get(class_name, {}).get('f1-score', 0) for r in improved_results]
        base_scores = [r['report'].get(class_name, {}).get('f1-score', 0) for r in baseline_results]
        improved_class_f1.append(np.mean(imp_scores))
        baseline_class_f1.append(np.mean(base_scores))
    
    x_classes = np.arange(len(classes))
    ax6.plot(x_classes, improved_class_f1, marker='o', linewidth=2.5, markersize=10, 
             label='Improved HMM', color='#2ecc71', linestyle='-')
    ax6.plot(x_classes, baseline_class_f1, marker='s', linewidth=2.5, markersize=10,
             label='Baseline HMM', color='#e74c3c', linestyle='--')
    ax6.set_xlabel('Signal Class', fontsize=12, fontweight='bold')
    ax6.set_ylabel('F1-Score', fontsize=12, fontweight='bold')
    ax6.set_title('📉 Per-Class F1-Score', fontsize=14, fontweight='bold', pad=15)
    ax6.set_xticks(x_classes)
    ax6.set_xticklabels(['S.Sell', 'Sell', 'Hold', 'Buy', 'S.Buy'], rotation=45, ha='right')
    ax6.legend(loc='best')
    ax6.grid(True, alpha=0.3, linestyle='--')
    ax6.set_ylim([0, 1])
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\n" + "="*70)
    print("📊 НЭГДСЭН ҮР ДҮН - SUMMARY STATISTICS")
    print("="*70)
    print(f"\n🟢 IMPROVED MODEL:")
    print(f"   Average Accuracy:  {avg_acc_imp:.2f}%")
    print(f"   Average F1-Score:  {avg_f1_imp:.3f}")
    print(f"\n🔴 BASELINE MODEL:")
    print(f"   Average Accuracy:  {avg_acc_base:.2f}%")
    print(f"   Average F1-Score:  {avg_f1_base:.3f}")
    print(f"\n📈 IMPROVEMENT:")
    print(f"   Accuracy:  {improvement_acc:+.2f}%")
    print(f"   F1-Score:  {improvement_f1:+.2f}%")
    print("="*70)

print("✅ Visualization function бэлэн")

✅ Visualization function бэлэн


### 🚀 Run Complete Evaluation Pipeline


In [22]:
# 1. Train baseline model
print("STEP 1: Baseline Model сургаж байна...\n")
baseline_artifacts = train_baseline_model('../data/train/EUR_USD_1min.csv')

print("\n" + "="*70)
print("✅ BASELINE MODEL БЭЛЭН!")
print("="*70)

STEP 1: Baseline Model сургаж байна...


📊 BASELINE HMM MODEL (Өмнөх хувилбар)

📂 Дата уншиж байна...
  ✅ 1,859,492 мөр уншигдлаа

🔧 Basic features тооцоолж байна...
  ✅ 1,859,492 мөр уншигдлаа

🔧 Basic features тооцоолж байна...

🏷️ Simple labels үүсгэж байна...

🏷️ Simple labels үүсгэж байна...

🔍 NaN/Inf утгууд шалгаж байна...
  ✅ Цэвэрлэсэн: 1,858,543 samples үлдлээ

📊 Dataset: 1,858,543 samples, 6 features

✂️ Split: Train=1,486,834, Test=371,709

⚖️ Scaling...

🔍 NaN/Inf утгууд шалгаж байна...
  ✅ Цэвэрлэсэн: 1,858,543 samples үлдлээ

📊 Dataset: 1,858,543 samples, 6 features

✂️ Split: Train=1,486,834, Test=371,709

⚖️ Scaling...

🎯 Simple HMM сургаж байна...

🎯 Simple HMM сургаж байна...

🎯 BASELINE TEST RESULTS:
  Test Accuracy:  17.01%
  Test F1-Score:  0.228

✅ BASELINE MODEL БЭЛЭН!

🎯 BASELINE TEST RESULTS:
  Test Accuracy:  17.01%
  Test F1-Score:  0.228

✅ BASELINE MODEL БЭЛЭН!


In [24]:
# 2. Test data files
test_files = [
    'EUR_USD_test.csv',
    'GBP_USD_test.csv',
    'USD_JPY_test.csv',
    'USD_CAD_test.csv',
    'USD_CHF_test.csv',
    'XAU_USD_test.csv'
]

print("\n" + "="*70)
print("STEP 2: Бүх Test Data дээр үнэлж байна...")
print("="*70)

# 3. Evaluate both models on all test files
improved_results = []
baseline_results = []

for i, test_file in enumerate(test_files, 1):
    test_path = f'../data/test/{test_file}'
    
    print(f"\n\n{'#'*70}")
    print(f"📊 TEST {i}/{len(test_files)}: {test_file}")
    print(f"{'#'*70}")
    
    # Improved model
    print("\n🟢 IMPROVED MODEL:")
    imp_result = evaluate_on_test_data(
        model_artifacts, 
        test_path, 
        model_name=f"Improved HMM - {test_file.replace('_test.csv', '')}"
    )
    improved_results.append(imp_result)
    
    # Baseline model
    print("\n🔴 BASELINE MODEL:")
    base_result = evaluate_on_test_data(
        baseline_artifacts,
        test_path,
        model_name=f"Baseline HMM - {test_file.replace('_test.csv', '')}"
    )
    baseline_results.append(base_result)
    
    # Quick comparison
    acc_diff = (imp_result['accuracy'] - base_result['accuracy']) * 100
    f1_diff = (imp_result['f1'] - base_result['f1']) * 100
    
    print(f"\n{'─'*70}")
    print(f"⚡ QUICK COMPARISON for {test_file.replace('_test.csv', '')}:")
    print(f"   Accuracy difference: {acc_diff:+.2f}%")
    print(f"   F1-Score difference: {f1_diff:+.2f}%")
    print(f"{'─'*70}")

print("\n\n" + "="*70)
print("✅ БҮХ TEST DATA ҮНЭЛЭГДЛЭЭ!")
print("="*70)


STEP 2: Бүх Test Data дээр үнэлж байна...


######################################################################
📊 TEST 1/6: EUR_USD_test.csv
######################################################################

🟢 IMPROVED MODEL:

🧪 Improved HMM - EUR_USD - TEST DATA ҮНЭЛГЭЭ

📂 Test data уншиж байна: EUR_USD_test.csv
  ✅ 296,778 мөр уншигдлаа

🔧 Features тооцоолж байна...
  ✅ 296,778 мөр уншигдлаа

🔧 Features тооцоолж байна...
✅ 45 features тооцоологдлоо

🏷️ Labels үүсгэж байна...
✅ 45 features тооцоологдлоо

🏷️ Labels үүсгэж байна...

📊 Label distribution:
  0 - Strong Sell    : 33,965 ( 11.5%)
  1 - Sell           : 28,307 (  9.5%)
  2 - Hold           : 172,401 ( 58.1%)
  3 - Buy            : 61,902 ( 20.9%)
  4 - Strong Buy     :      0 (  0.0%)

📊 Test data shape: (296575, 24)

🔮 Таамаглал хийж байна...

📊 Label distribution:
  0 - Strong Sell    : 33,965 ( 11.5%)
  1 - Sell           : 28,307 (  9.5%)
  2 - Hold           : 172,401 ( 58.1%)
  3 - Buy            : 61,902 ( 20

KeyError: "['atr', 'ma_cross', 'rsi'] not in index"

In [ ]:
# 4. Plot comprehensive comparison
print("\n\n" + "="*70)
print("STEP 3: Харьцуулалтын график зурж байна...")
print("="*70)

plot_comparison_results(improved_results, baseline_results, test_files)

### 📊 Detailed Per-Pair Confusion Matrices


In [ ]:
# Detailed confusion matrices for each currency pair
fig, axes = plt.subplots(3, 4, figsize=(20, 15))
axes = axes.flatten()

label_names = ['S.Sell', 'Sell', 'Hold', 'Buy', 'S.Buy']

for i, (test_file, imp_res, base_res) in enumerate(zip(test_files, improved_results, baseline_results)):
    pair_name = test_file.replace('_test.csv', '')
    
    # Improved model confusion matrix
    ax_imp = axes[i*2]
    sns.heatmap(imp_res['confusion_matrix'], annot=True, fmt='d', cmap='Greens',
                xticklabels=label_names, yticklabels=label_names,
                cbar_kws={'label': 'Count'}, ax=ax_imp)
    ax_imp.set_title(f'🟢 {pair_name} - Improved\nAcc: {imp_res["accuracy"]*100:.1f}%', 
                     fontsize=12, fontweight='bold')
    ax_imp.set_ylabel('True', fontsize=10, fontweight='bold')
    ax_imp.set_xlabel('Predicted', fontsize=10, fontweight='bold')
    
    # Baseline model confusion matrix
    ax_base = axes[i*2 + 1]
    sns.heatmap(base_res['confusion_matrix'], annot=True, fmt='d', cmap='Reds',
                xticklabels=label_names, yticklabels=label_names,
                cbar_kws={'label': 'Count'}, ax=ax_base)
    ax_base.set_title(f'🔴 {pair_name} - Baseline\nAcc: {base_res["accuracy"]*100:.1f}%', 
                      fontsize=12, fontweight='bold')
    ax_base.set_ylabel('True', fontsize=10, fontweight='bold')
    ax_base.set_xlabel('Predicted', fontsize=10, fontweight='bold')

plt.suptitle('📊 CONFUSION MATRICES - ALL CURRENCY PAIRS', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("\n✅ Бүх currency pair-ын confusion matrices харуулагдлаа!")

### 📈 Final Summary Report


In [ ]:
# Create detailed summary table
import pandas as pd

summary_data = []

for test_file, imp_res, base_res in zip(test_files, improved_results, baseline_results):
    pair = test_file.replace('_test.csv', '')
    
    summary_data.append({
        'Currency Pair': pair,
        'Improved Acc (%)': f"{imp_res['accuracy']*100:.2f}",
        'Baseline Acc (%)': f"{base_res['accuracy']*100:.2f}",
        'Acc Improvement': f"{(imp_res['accuracy'] - base_res['accuracy'])*100:+.2f}%",
        'Improved F1': f"{imp_res['f1']:.3f}",
        'Baseline F1': f"{base_res['f1']:.3f}",
        'F1 Improvement': f"{(imp_res['f1'] - base_res['f1']):.3f}"
    })

summary_df = pd.DataFrame(summary_data)

# Add average row
avg_row = {
    'Currency Pair': '📊 AVERAGE',
    'Improved Acc (%)': f"{np.mean([r['accuracy']*100 for r in improved_results]):.2f}",
    'Baseline Acc (%)': f"{np.mean([r['accuracy']*100 for r in baseline_results]):.2f}",
    'Acc Improvement': f"{np.mean([(i['accuracy'] - b['accuracy'])*100 for i, b in zip(improved_results, baseline_results)]):+.2f}%",
    'Improved F1': f"{np.mean([r['f1'] for r in improved_results]):.3f}",
    'Baseline F1': f"{np.mean([r['f1'] for r in baseline_results]):.3f}",
    'F1 Improvement': f"{np.mean([i['f1'] - b['f1'] for i, b in zip(improved_results, baseline_results)]):.3f}"
}
summary_df = pd.concat([summary_df, pd.DataFrame([avg_row])], ignore_index=True)

print("\n" + "="*100)
print("📊 ЭЦСИЙН ТАЙЛАН - FINAL SUMMARY REPORT")
print("="*100)
print(summary_df.to_string(index=False))
print("="*100)

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

# Accuracy table
acc_data = summary_df[['Currency Pair', 'Improved Acc (%)', 'Baseline Acc (%)', 'Acc Improvement']].iloc[:-1]
colors = [['#e8f5e9' if i % 2 == 0 else '#ffffff'] * 4 for i in range(len(acc_data))]

ax1.axis('tight')
ax1.axis('off')
table1 = ax1.table(cellText=acc_data.values,
                   colLabels=acc_data.columns,
                   cellLoc='center',
                   loc='center',
                   cellColours=colors,
                   colColours=['#4caf50']*4)
table1.auto_set_font_size(False)
table1.set_fontsize(10)
table1.scale(1, 2)
ax1.set_title('🎯 ACCURACY COMPARISON', fontsize=14, fontweight='bold', pad=20)

# F1-Score table
f1_data = summary_df[['Currency Pair', 'Improved F1', 'Baseline F1', 'F1 Improvement']].iloc[:-1]
colors = [['#e3f2fd' if i % 2 == 0 else '#ffffff'] * 4 for i in range(len(f1_data))]

ax2.axis('tight')
ax2.axis('off')
table2 = ax2.table(cellText=f1_data.values,
                   colLabels=f1_data.columns,
                   cellLoc='center',
                   loc='center',
                   cellColours=colors,
                   colColours=['#2196f3']*4)
table2.auto_set_font_size(False)
table2.set_fontsize(10)
table2.scale(1, 2)
ax2.set_title('📊 F1-SCORE COMPARISON', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

# Final verdict
avg_acc_improvement = np.mean([(i['accuracy'] - b['accuracy'])*100 for i, b in zip(improved_results, baseline_results)])
avg_f1_improvement = np.mean([i['f1'] - b['f1'] for i, b in zip(improved_results, baseline_results)])

print("\n\n" + "🏆"*35)
print("                    ЭЦСИЙН ДҮГНЭЛТ - FINAL VERDICT")
print("🏆"*35)

if avg_acc_improvement > 5:
    verdict = "🌟 МАШ САЙН САЙЖИРСАН! Improved model нь өмнөх model-оос хамаагүй сайн!"
elif avg_acc_improvement > 2:
    verdict = "✅ САЙЖИРСАН! Improved model нь тодорхой сайжрал харуулж байна."
elif avg_acc_improvement > 0:
    verdict = "⚠️ БАГА ЗЭРЭГ САЙЖИРСАН. Одоогийн сайжруулалт тийм ч их биш."
else:
    verdict = "❌ МУУДСАН. Энэ нь тохиолдох боломжтой, hyperparameter-ийг засах хэрэгтэй."

print(f"\n{verdict}")
print(f"\n📈 Average Accuracy Improvement: {avg_acc_improvement:+.2f}%")
print(f"📈 Average F1-Score Improvement: {avg_f1_improvement:+.3f}")

print("\n" + "🏆"*35)
print("\n✅ БҮХ ҮНЭЛГЭЭ ДУУСЛАА!")